### NECESSARY EXPLANATIONS

i had to create another model that passed the concatanted model because i could not train the model on a layer only that had the outputs so i made a final model with only a one dense layer that passed the concatanated layer as input and the output layer with 2 neurons and the necessary activation function as the output for the final model. that is the final output layer that i consider for my model.

### LOADING NECESSARY LIBRARIES

In [ ]:
!pip install tensorflow==2.x

ERROR: Could not find a version that satisfies the requirement tensorflow==2.x (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1)
ERROR: No matching distribution found for tensorflow==2.x


In [ ]:
!pip install keras

In [ ]:
! pip install scikeras
from scikeras.wrappers import KerasClassifier


In [ ]:
import urllib.request
import zipfile
import sklearn
import numpy as np
import pandas as pd
import matplotlib as mpl
import scipy as sp
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
import sklearn
import numpy as np
import pandas as pd
import matplotlib as mpl
import scipy as sp
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import seaborn as sns
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.metrics import roc_curve, precision_recall_curve, precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score, matthews_corrcoef, confusion_matrix
import imblearn
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, TomekLinks
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold, StratifiedKFold, RepeatedKFold, ShuffleSplit, StratifiedShuffleSplit, learning_curve, validation_curve
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve
from sklearn.dummy import DummyClassifier
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer, f1_score
import sklearn
import numpy as np
import pandas as pd
import matplotlib as mpl
import scipy as sp
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder, OneHotEncoder, KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from tensorflow.keras.regularizers import l1
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

LOADING THE DATA

In [ ]:

# ZIP data URL from GitHub
zip_url = "https://github.com/vitalunax/examdnn/blob/main/input%20(3).zip?raw=true"

# Download the ZIP
zip_path, _ = urllib.request.urlretrieve(zip_url)

# Unzip the folder
target_folder = "temp"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(target_folder)

data_path = f"{target_folder}/input.pkl"

In [ ]:
import pickle as pk

# Open the pickle data
with open(data_path, 'rb') as f:
  train_labels, test_labels, train_texts, test_texts, train_bow, test_bow = pk.load(f)


### PREPROCESSING DATA AND PREPARING INPUT


> Q2



In [ ]:
from tensorflow.keras.callbacks import EarlyStopping


 the labels are one hot encoded to be used in training.

In [ ]:
# One-hot encode the labels
encoder = OneHotEncoder(sparse=False)
train_labels_ohe = encoder.fit_transform(np.array(train_labels).reshape(-1, 1))
test_labels_ohe = encoder.fit_transform(np.array(test_labels).reshape(-1, 1))
test_labels_ohe.shape

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(140, 2)

here i defined early stopping because the grid search and training took a lot of time and put a  lot of weight on the hardware

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=3,          # Number of epochs with no improvement after which training will be stopped
    verbose=1,           # Log when training is stopped
    mode='min',          # The direction is automatically inferred if not set, but 'min' means we want to minimize validation loss
    restore_best_weights=True  # Restores model weights from the epoch with the minimum validation loss
)


here we have the text data.I did pre processing by first tokenizing and then padding to later on make the model learn the word embeddings and then I scaled it

In [ ]:
# Preprocessing for text data

max_words = 10000
embedding_dim = 300
max_len = 1302
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)
train_sequences = tokenizer.texts_to_sequences(train_texts)
train_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post', truncating='post')
test_sequences = tokenizer.texts_to_sequences(test_texts)
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post', truncating='post')


In [ ]:
    max_len_text_data = 1302  # Adjust this based on your actual data


## MODEL STRUCTURING



> Q2
Q5
Q3

Here we have two different inputs in which I use both of them with the labels for their own intended model pipe line

Here it is also visable that i have used the bianry cross entropy loss




In [ ]:
def create_model1(optimizer='adam', activation='ReLU',out_act='softmax', loss='binary_crossentropy', batch_size=32, learning_rate=0.001, initializer='glorot_uniform', weight_reg=l1(0.01)):
    # Define your Keras model architecture here
    max_len_text_data = 1302  # Adjust this based on your actual data
    model1 = Sequential()
    model1.add(Input(shape=(max_len_text_data,)))
    model1.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len_text_data))
    model1.add(Flatten())
    model1.add(Dense(128, activation=activation, kernel_initializer=initializer, kernel_regularizer=weight_reg))
    model1.add(Dense(64, activation=activation, kernel_initializer=initializer, kernel_regularizer=weight_reg))
    model1.add(Dense(2, activation=out_act))

    model1.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

    return model1

BOW DATA


> Q2
Q5


here we have BOW data which I applied smoothing to deal with the problem of large amount of zero values after smoothing i applied scaling to ensure that the values are between the needed values

In [ ]:
input_size_bow_frequency = 12854
# Define the smoothing factor (you can adjust this based on your needs)
smoothing_factor = 1e-5

# Scale the BoW vectors using Min-Max scaling
scaler = MinMaxScaler()
train_bow_scaled = scaler.fit_transform(train_bow)
test_bow_scaled = scaler.transform(test_bow)

# Apply additive smoothing to handle zero-values
train_bow_smoothed = train_bow_scaled + smoothing_factor
test_bow_smoothed = test_bow_scaled + smoothing_factor

# Ensure that the values are within the desired range after smoothing
train_bow_smoothed = np.clip(train_bow_smoothed, 0, 1)
test_bow_smoothed = np.clip(test_bow_smoothed, 0, 1)



> Q2 Q5 Q3



Here you can see that i HAVE CREATED TWO DIFFERENT MODELS WITH SEVERAL FULLY CONNECTED DENDSE LAYERS WITH RELU ACTIVATION FUNCTION AND AS A FINAL LAYER TWO NEURONS AND SOFMAX ACTIVATION FUNCTION.

As an initilizer I used xavier glorot and as regulariser L1 to deal with the sparsity problem and

you can see that i have defined each model as their own pipeline

and also i have used binary cross entropy as loss function

In [ ]:
def create_model2(optimizer='adam', activation='ReLU',out_act='softmax', loss='binary_crossentropy', batch_size=32, learning_rate=0.001, initializer='glorot_uniform', weight_reg=l1(0.01)):


  # Define the size of BoW frequency data
  input_size_bow_frequency = 12854

  # Create a Sequential model for BoW frequency data
  model2 = Sequential()
  model2.add(Input(shape=(input_size_bow_frequency,)))
  model2.add(Dense(128, activation=activation, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=l1(0.01)))
  model2.add(Dense(64, activation=activation, kernel_initializer='glorot_uniform', bias_initializer='zeros',kernel_regularizer=l1(0.01)))
  model2.add(Dense(2, activation=out_act))  # Final layer before the merge



  # Compile the merged model
  model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=10**-2,
                                                 clipvalue=0.5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

  return model2



> Q5
Q4
Q3


OUTPUT LAYER
LOSS FUNCTION



after creating the 2 separate models i have created a one last model with 1 dense layer taking the concateanted layer as input and passing them through to later on use it in training and grid search. and here the only layer that is my intended output layer has again 2 neurons with softmax activation that will have the merged probabilites of the 2 models in the concatanted layer.

you can also see the loss function as binary cross entropy here

In [ ]:
def create_model(optimizer='adam', activation='relu', loss='binary_crossentropy', learning_rate=0.01, neurons=2, layers=1):
  model1 = create_model1(learning_rate=0.01, loss='binary_crossentropy', activation='ReLU', out_act='softmax',  weight_reg=l1(0.01))
  model2 = create_model2(learning_rate=0.01, loss='binary_crossentropy', activation='ReLU', out_act='softmax', weight_reg=l1(0.01))
  concatenated_layer = Concatenate(name='concatenated_layer')([model1.layers[-1].output, model2.layers[-1].output])
  output = Dense(2, activation='softmax')(concatenated_layer)  # Define the appropriate shape and activatio
  model1_input = model1.layers[0].input
  model2_input = model2.layers[0].input
  model = Model(inputs=[model1_input, model2_input], outputs=output)
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
  return model

In [ ]:
model = create_model()
model


In [ ]:
print(type(train_padded))
print(type(train_bow_smoothed))
print(type(train_labels_ohe))


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>




> Q6





>Q5

TRAINING THE MODEL ON THE SELECTED HYPERPARAMETERS



here i trained my model and made predictions on it. i have only 30 epochs because again it took a lot of time. I have defined batchsize validation split and 30 epochs.

In [ ]:
# Training
model.fit([train_padded, train_bow_smoothed], train_labels_ohe,
    validation_split=0.2,  # Assuming you have a validation split
    epochs=30,  # Set to a higher number since early stopping will halt training
    callbacks=[early_stopping],
    batch_size=32,  # Example batch size, adjust according to your needs
    verbose=1
)



Epoch 1/30
14/14 [==============================] - 51s 3s/step - loss: 434.2068 - accuracy: 0.5692 - val_loss: 213.5183 - val_accuracy: 0.2232
Epoch 2/30
14/14 [==============================] - 30s 2s/step - loss: 152.1018 - accuracy: 0.6295 - val_loss: 121.3521 - val_accuracy: 0.7768
Epoch 3/30
14/14 [==============================] - 30s 2s/step - loss: 99.8573 - accuracy: 0.6496 - val_loss: 85.2351 - val_accuracy: 0.7768
Epoch 4/30
14/14 [==============================] - 29s 2s/step - loss: 81.0196 - accuracy: 0.6942 - val_loss: 76.0946 - val_accuracy: 0.7768
Epoch 5/30
14/14 [==============================] - 32s 2s/step - loss: 73.7392 - accuracy: 0.6942 - val_loss: 74.4379 - val_accuracy: 0.7768
Epoch 6/30
14/14 [==============================] - 31s 2s/step - loss: 69.7655 - accuracy: 0.6942 - val_loss: 69.2735 - val_accuracy: 0.7768
Epoch 7/30
14/14 [==============================] - 30s 2s/step - loss: 66.0965 - accuracy: 0.6942 - val_loss: 63.6823 - val_accuracy: 0.7768
Ep

In [ ]:
# Prediction
predictions = model.predict([test_padded, test_bow_smoothed])

5/5 [==============================] - 1s 120ms/step


here i printed out the shapes for debugging reasons

In [ ]:
print("Shape of test_padded:", test_padded.shape)
print("Shape of test_bow_smoothed:", test_bow_smoothed.shape)
print("Shape of test_labels_ohe:", test_labels_ohe.shape)


Shape of test_padded: (140, 1302)
Shape of test_bow_smoothed: (140, 12854)
Shape of test_labels_ohe: (140, 2)


here i wrapped the model in keras classifier to perform grid search later on

In [ ]:
# Wrap the model so it can be used by scikit-learn
model = KerasClassifier(build_fn=create_model, verbose=0)
model

KerasClassifier(
	model=None
	build_fn=<function create_model at 0x7d8ba266e440>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	class_weight=None
)

### MODEL SELECTION


hyperparameter tuning and model selection



> Q5 Q6

i couldnt perform grid search on the combined model so i performed it on separately on each model and then extracted the best model out of those.

the hyperparameters are the ones included in the create model functions. the initializers and regularizers are also visable.

In [ ]:
from scikeras.wrappers import KerasClassifier

model1_clf = KerasClassifier(activation='relu',learning_rate=0.001, model=create_model1, verbose=0)
model2_clf = KerasClassifier(activation='relu',learning_rate=0.001, model=create_model2, verbose=0)
param_grid = {
    'optimizer': ['adam', 'sgd'],
    'activation': ['relu', 'sigmoid'],
    'loss': ['binary_crossentropy'],
    'learning_rate': [0.001, 0.01],
    'batch_size': [32, 64],
}


In [ ]:

# Grid search for model1
grid1 = GridSearchCV(estimator=model1_clf, param_grid=param_grid, n_jobs=-1, cv=3, scoring='accuracy')
grid_result1 = grid1.fit(test_padded, test_labels_ohe)  # Assuming X_train, y_train are defined


In [ ]:
best_model1 = grid1.best_estimator_
best_accuracy1 = grid1.best_score_
print(f"Best Accuracy (Validation) for Model 1: {best_accuracy1}")


Best Accuracy (Validation) for Model 1: 0.72895467160037


In [ ]:

# Grid search for model2
grid2 = GridSearchCV(estimator=model2_clf, param_grid=param_grid, n_jobs=-1, cv=3, scoring='accuracy')
grid_result2 = grid2.fit(test_bow_smoothed, test_labels_ohe)


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [ ]:
best_model2 = grid2.best_estimator_
best_accuracy2 = grid2.best_score_
print(f"Best Accuracy (Validation) for Model 2: {best_accuracy2}")


Best Accuracy (Validation) for Model 2: 0.72895467160037


here i printed the evaluation metrics which seem completely off which i assume to be because of the complexity of the model i preferred. and i believe i have some other problems that i couldnt seem to figure out why. but as a result i have made predictions on the test data to evaluate the generalization capabilities of my model.

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Assuming 'X_test', 'y_test' is your test dataset
y1_pred = best_model1.predict(test_padded)

print("Accuracy:", accuracy_score(test_labels_ohe, y1_pred))
print("Classification Report:\n", classification_report(test_labels_ohe, y1_pred))


Accuracy: 0.7285714285714285
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.73      1.00      0.84       102

   micro avg       0.73      0.73      0.73       140
   macro avg       0.36      0.50      0.42       140
weighted avg       0.53      0.73      0.61       140
 samples avg       0.73      0.73      0.73       140



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Assuming 'X_test', 'y_test' is your test dataset
y2_pred = best_model2.predict(test_bow_smoothed)

print("Accuracy:", accuracy_score(test_labels_ohe, y2_pred))
print("Classification Report:\n", classification_report(test_labels_ohe, y2_pred))


Accuracy: 0.7285714285714285
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.73      1.00      0.84       102

   micro avg       0.73      0.73      0.73       140
   macro avg       0.36      0.50      0.42       140
weighted avg       0.53      0.73      0.61       140
 samples avg       0.73      0.73      0.73       140



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
